# Dance Diffusion finetune v0.2

Licensed under the MIT License

Copyright (c) 2022 Zach Evans

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [ ]:
#@title <- View Changelog
skip_for_run_all = False #@param {type: 'boolean'}

if skip_for_run_all == False:
  print(
      '''
  v0.1 Update: Jul 30, 2022 - zqevans
      
      - Added Dance Diffusion finetune notebooks

  v0.2 Update: Oct 29, 2022 - twobob
      
      - Ported to flavioschneiders repo in the style of Harmonai
      added changelog, experiment editor and .env generator
      '''
      )

In [ ]:
#@title Check GPU Status
skip_for_run_all = False #@param {type: 'boolean'}

import subprocess
if skip_for_run_all == False:
  simple_nvidia_smi_display = True#@param {type:"boolean"}
  if simple_nvidia_smi_display:
      #!nvidia-smi
      nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(nvidiasmi_output)
  else:
      #!nvidia-smi -i 0 -e 0
      nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(nvidiasmi_output)
      nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(nvidiasmi_ecc_note)

In [103]:
#@title Cleardown options
import os
import time
import datetime 
from IPython.display import clear_output  
from os.path import exists
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import HBox, Label
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
clear_logs_after_setup = True#@param {type:"boolean"}

In [ ]:
try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    is_kaggle = False
    #@markdown Check to connect your Google Drive
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")
    is_kaggle = True if os.path.exists('/kaggle/working') else False
if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        ai_root = '/content/drive/MyDrive/AI'
        root_path = f'{ai_root}/Dance_Diffusion'
    else:
        root_path = '/content'
    true_root = '/content'    
else:
    root_path = '/kaggle/working/' if is_kaggle else os.getcwd()
    true_root = '/kaggle/working/' if is_kaggle else os.getcwd()

In [ ]:
#@title Setup 
now = datetime.datetime.now()
def done(clear=False):
    now = datetime.datetime.now()
    if not clear:
        clear_output(wait=False)    
    print(f"done at {now.hour}:{now.minute}:{now.second}")

import os
!pip install ipywidgets
#!pip uninstall -y audio-diffusion-pytorch-trainer
#!rm -rf /content/audio-diffusion-pytorch-trainer/
os.chdir(true_root)
#!git clone --branch NoLog https://github.com/twobob/audio-diffusion-pytorch-trainer
!git clone https://github.com/archinetai/audio-diffusion-pytorch-trainer
os.chdir(f'{true_root}/audio-diffusion-pytorch-trainer')
!pip install -r requirements.txt
if clear_logs_after_setup:
  done()
print('archinetai/audio-diffusion-pytorch-trainer install completed')

In [180]:
from prompt_toolkit.layout import layout
#@title Choose An Experiment
import ipywidgets as widgets
from ipywidgets import HBox, Label
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
import time
import pandas as pd
out = widgets.Output()

df = df = pd.DataFrame(columns = ['Dropdown_column', 'Float_column'])

def list_files(your_directory):
  return sorted( filter( lambda x: os.path.isfile(os.path.join(your_directory, x)), os.listdir(your_directory) ) )

expdir=f'{true_root}/audio-diffusion-pytorch-trainer/exp/'
archdir=f'{true_root}/audio-diffusion-pytorch-trainer/exp/archived/'
current_dyamic_dir = expdir # presumes LIVE as default

files = list_files(expdir)
archives =  list_files(archdir)

isYouTube = False

def exp_loader(fully_qualified_filepath):
  global isYouTube
  with open(fully_qualified_filepath, 'r') as doc:
    chomp = doc.read()
    isYouTube = True if 'YoutubeDataset' in chomp or 'CommonVoiceDataset' in chomp else False
    return chomp

def exp_writer(fully_qualified_filepath, string_text):
  global isYouTube
  with open(fully_qualified_filepath, 'w') as doc:
    isYouTube = True if 'YoutubeDataset' in string_text or 'CommonVoiceDataset' in string_text else False
    doc.write(string_text)

def get_button_layout():
  return Layout(
    display='inline-flex',
    flex_flow='row',
    justify_content='center',
    padding = '5%',
    align_items = 'flex-start'
)

def dropdown_handler(change):
    global drop_down_input
    drop_down_input = change.new
    if change['type'] == 'change': dependent_drop_down.options=dependent_drop_down_elements[change.new]
    current_dyamic_dir = expdir if drop_down.value=='LiveExperiments' else archdir
    

def dep_dropdown_handler(change):
    global EXPERIMENT
    current_dyamic_dir = expdir if drop_down.value=='LiveExperiments' else archdir
    editor.value = exp_loader(f'{current_dyamic_dir}{dependent_drop_down.value}')
    EXPERIMENT = dependent_drop_down.value.replace('.yaml','')

def on_edit_button_clicked(b):
    editor.value = exp_loader(f'{current_dyamic_dir}/{dependent_drop_down.value}')
    editor.layout.display = "flex"
    editbutton.layout.display = "none"
    savebutton.layout.display = "flex"

def on_save_button_clicked(b):
    global df
    with out:
      exp_writer(f'{current_dyamic_dir}/{dependent_drop_down.value}', editor.value)
      editor.layout.display = "none"
      savebutton.layout.display = "none"
      editbutton.layout.display = "flex"

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

edit_button_item_layout = get_button_layout()

button_item_layout = get_button_layout()
button_item_layout.display = "none"

#setup the default loaded experiment, hidden in the editor
doctxt= exp_loader(f'{current_dyamic_dir}/{files[0]}')

drop_down_input = 'LiveExperiments'
drop_down = widgets.Dropdown(options=('LiveExperiments', 'ArchivedExperiments'),disabled=True) # Toggle for arch view

dependent_drop_down_elements = {}
dependent_drop_down_elements['LiveExperiments'] = files
dependent_drop_down_elements['ArchivedExperiments'] = archives
dependent_drop_down = widgets.Dropdown(options=(dependent_drop_down_elements['LiveExperiments']))

drop_down.observe(dropdown_handler, names='value')
dependent_drop_down.observe(dep_dropdown_handler, names='value')

savebutton = widgets.Button(description='save experiment', layout=button_item_layout)
savebutton.on_click(on_save_button_clicked)

editbutton = widgets.Button(description='edit experiment', layout = edit_button_item_layout)
editbutton.on_click(on_edit_button_clicked)

editor  = widgets.Textarea(
    value=doctxt,
    placeholder='Edit Here',
    description='Experiment:',
    disabled=False,
    layout=Layout(
    display='none',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%',
    height='800px',
    padding = '1%'
)
)

form_items = [         
    Box([Label(value='Experiment Editor'),
         drop_down], layout=form_item_layout),
    Box([Label(value='Choose an experiment to edit then run'),
         dependent_drop_down], layout=form_item_layout),
         editor,
         editbutton,
         savebutton
        ]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='99%',
    padding = '1%'
))
display(form)
display(out)
EXPERIMENT = dependent_drop_down.value.replace('.yaml','')

Box(children=(Box(children=(Label(value='Experiment Editor'), Dropdown(options=('LiveExperiments', 'ArchivedEx…

Output()

In [ ]:
#TMPENV = f'{true_root}/audio-diffusion-pytorch-trainer/.env.tmp'
LIVEENV = f'{true_root}/audio-diffusion-pytorch-trainer/.env'
!cp $TMPENV $LIVEENV
#@title Setup paths, logins and logging
#@markdown Setup the .env file options
custom_log_path = 'logs'#@param {type: 'string'}
custom_data_path = 'data'#@param {type: 'string'}
WANDB_PROJECT = 'test_project'#@param {type: 'string'}
#@markdown weights and biases username
#@markdown Required if using wandb logger
WANDB_ENTITY = 'YOURNAME'#@param {type: 'string'}
#@markdown weights and biases API key
WANDB_API_KEY = 'YOURKEY'#@param {type: 'string'}
#@markdown huggingface API key
#@markdown Required if using Common Voice dataset
HUGGINGFACE_TOKEN = 'YOURTOKEN'#@param {type: 'string'}

file = open(LIVEENV, "r")
replacement = ""

# using the for loop
for count, line in enumerate(file):
    line = line.strip()  
    if 0 == count:
      line=  changes = f"DIR_LOGS=/{custom_log_path}"
    if 1 == count:
      line= changes = f"DIR_DATA=/{custom_data_path}"
      
    changes = line.replace("WANDB_PROJECT=wandbprojectname", f"WANDB_PROJECT={WANDB_PROJECT}")  
    changes = changes.replace("WANDB_ENTITY=wandbuser", f"WANDB_ENTITY={WANDB_ENTITY}")  
    changes = changes.replace("WANDB_API_KEY=wandbapikey", f"WANDB_API_KEY={WANDB_API_KEY}")  
    changes = changes.replace("HUGGINGFACE_TOKEN=huggingfacetoken", f"HUGGINGFACE_TOKEN={HUGGINGFACE_TOKEN}")  
    replacement = replacement + changes + "\n"
with open(LIVEENV, 'w') as f:
    f.write(replacement)
f.close() 
#os.chdir(true_root)
if clear_logs_after_setup:
  done()
print('options set')

In [ ]:
%env  HYDRA_FULL_ERROR=0
TRAIN_CMD = f'{true_root}/audio-diffusion-pytorch-trainer/train.py'
EXPERIMENT_CMD = f' exp={EXPERIMENT}'
PATH_CMD = '' if isYouTube else f'++datamodule.dataset.path={true_root}/'
#@title Run the finetune
## n GPU TRAINING with added data dir.

os.chdir(true_root)
!python $TRAIN_CMD $EXPERIMENT_CMD trainer.gpus=1 $PATH_CMD
